In [10]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models

2024-06-03 09:35:29.147994: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
def getSplittedDataset(x, y, norm=False):
    X_train_sub, X_test , y_train_sub, y_test = train_test_split(
        x, y, test_size=0.10, random_state=42)
    X_train, X_test_val, y_train, y_test_val = train_test_split(
        X_train_sub, y_train_sub, test_size=0.20, random_state=42)
    if (norm):
        X_train, X_test_val, X_test = X_train / 255.0, X_test_val / 255.0, X_test / 255.0
    return X_train, X_test_val, X_test, y_train, y_test_val, y_test

#X_train, X_test_val, X_test, y_train, y_test_val, y_test = getSplittedDataset(norm=True)

def cnn(input_shape, num_clases):
    model = Sequential()
    #1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    #second
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    #third
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
     
    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))

    #output
    model.add(Dense(num_clases, activation='softmax'))


    model.compile(optimizer=Adam(),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


    model.summary()
    
    return model


def training(model, X_train, X_test_val, X_test, 
            y_train, y_test_val, y_test , epoch,
            name_file ):
    history = model.fit(X_train, y_train , epochs=epoch,
                    validation_data=(X_test_val , y_test_val),
                    batch_size=64)
    #h5 format
    #model.save("firstTry.h5")
    model.save(name_file)
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f'Test accuracy: {test_acc} FOR: {name_file}')

def loadImages(inputRoute, df, objectiveColumn):
    totalImages = []
    for index, row in df.iterrows():
        uuid = row["UUID"]
        routeImage =  f"{inputRoute}/{uuid}.jpg"
        image = cv2.imread(routeImage)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        totalImages.append(image_rgb)

    imagesLoaded = np.array(totalImages)
    types = df[objectiveColumn].to_numpy()
    return imagesLoaded, types



In [42]:
df = pd.read_csv("prod_data_images_v3.csv")
key_value_map_categories = dict(zip(df['category_id'], df['category']))
key_value_map_categories

{0: 'microcontrolador',
 1: 'sensor',
 2: 'rele',
 3: 'servomotor',
 4: 'capacitor',
 5: 'resistencia',
 6: 'diodo',
 7: 'transistor'}

In [44]:
finalJson = {}
import json
keys_list = list(key_value_map_categories.keys())
for keyCategory, strValue  in key_value_map_categories.items():
    newTempDf = df[df['category_id'] == keyCategory]
    newTempDf['type_id_norm'] , mapOfTypes = pd.factorize(newTempDf['type_id'])
     
    if(len(mapOfTypes) > 1):
        ## vamo a hacer el modelo
        key_value_map_types = dict(zip(newTempDf['type_id'], newTempDf['type']))

        print(key_value_map_types)
        key_value_map_types = dict(zip(newTempDf['type_id_norm'], newTempDf['type']))

        print(key_value_map_types)
        finalJson[keyCategory] = {"name": strValue, "types":key_value_map_types }
with open('dataKeys.json', 'w') as file:
    json.dump(finalJson, file)

{0: 'ESP32', 5: 'MODULO BLUETOOTH', 6: 'MODULO BUZZER', 8: 'BREADBOARD POWER SUPPLY MODULE', 9: 'raspberry_pi', 58: 'stm32', 59: 'arduino_atmega'}
{0: 'ESP32', 1: 'MODULO BLUETOOTH', 2: 'MODULO BUZZER', 3: 'BREADBOARD POWER SUPPLY MODULE', 4: 'raspberry_pi', 5: 'stm32', 6: 'arduino_atmega'}
{1: 'SENSOR PIR', 2: 'SENSOR DE TEMPERATURA Y HUMEDAD', 10: 'sensor_angulo', 11: 'sensor_encoder'}
{0: 'SENSOR PIR', 1: 'SENSOR DE TEMPERATURA Y HUMEDAD', 2: 'sensor_angulo', 3: 'sensor_encoder'}
{3: 'RELE ESTADO SOLIDO', 4: 'RELE', 26: 'rele_1', 27: 'rele_2', 28: 'rele_3'}
{0: 'RELE ESTADO SOLIDO', 1: 'RELE', 2: 'rele_1', 3: 'rele_2', 4: 'rele_3'}
{12: 'cap_3300uf', 13: 'cap_220uf', 14: 'cap_2200uf', 15: 'cap_10uf', 16: 'cap_22uf', 17: 'cap_101', 18: 'cap_473', 19: 'cap_470uf', 20: 'cap_1000uf', 21: 'cap_330uf', 22: 'cap_33uf', 23: 'cap_581', 24: 'cap_1uf', 25: 'cap_47uf'}
{0: 'cap_3300uf', 1: 'cap_220uf', 2: 'cap_2200uf', 3: 'cap_10uf', 4: 'cap_22uf', 5: 'cap_101', 6: 'cap_473', 7: 'cap_470uf', 8:

/tmp/ipykernel_633730/869910476.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newTempDf['type_id_norm'] , mapOfTypes = pd.factorize(newTempDf['type_id'])
/tmp/ipykernel_633730/869910476.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newTempDf['type_id_norm'] , mapOfTypes = pd.factorize(newTempDf['type_id'])
/tmp/ipykernel_633730/869910476.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [22]:
keys_list = list(key_value_map_categories.keys())
for keyCategory in keys_list:
    newTempDf = df[df['category_id'] == keyCategory]
    newTempDf['type_id_norm'] , mapOfTypes = pd.factorize(newTempDf['type_id'])
     
    if(len(mapOfTypes) > 1):
        ## vamo a hacer el modelo
        key_value_map_types = dict(zip(newTempDf['type_id'], newTempDf['type']))
        x, y = loadImages(inputRoute = "images_final_reduced_100", 
                         df = newTempDf , objectiveColumn = "type_id_norm")
        X_train, X_test_val, X_test, y_train, y_test_val, y_test = getSplittedDataset(x = x , y = y, norm=True)
        model = cnn(input_shape=(100,100,3), num_clases=len(key_value_map_types) )
        training(model=model, X_train=X_train, X_test_val= X_test_val, X_test=X_test, 
                y_train= y_train, y_test_val=y_test_val, y_test=y_test, 
                epoch= 10, name_file=f"modelForCategory_{key_value_map_categories[keyCategory]}.h5")

/tmp/ipykernel_633730/3943309202.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newTempDf['type_id_norm'] , mapOfTypes = pd.factorize(newTempDf['type_id'])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 49, 49, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 23, 23, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 10, 10, 128)     

/tmp/ipykernel_633730/3943309202.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newTempDf['type_id_norm'] , mapOfTypes = pd.factorize(newTempDf['type_id'])


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 49, 49, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 23, 23, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 10, 10, 128)     

/tmp/ipykernel_633730/3943309202.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newTempDf['type_id_norm'] , mapOfTypes = pd.factorize(newTempDf['type_id'])


...

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 49, 49, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 23, 23, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 10, 10, 128)     

/tmp/ipykernel_633730/3943309202.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newTempDf['type_id_norm'] , mapOfTypes = pd.factorize(newTempDf['type_id'])


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 49, 49, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 23, 23, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 10, 10, 128)     

/tmp/ipykernel_633730/3943309202.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newTempDf['type_id_norm'] , mapOfTypes = pd.factorize(newTempDf['type_id'])


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 49, 49, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 23, 23, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_32 (Conv2D)          (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 10, 10, 128)    